In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms, models
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import sys, os
from glob import glob
import imageio
import pandas as pd

In [2]:
!wget -nc https://archive.ics.uci.edu/ml/machine-learning-databases/00278/SEMG_DB1.rar

--2023-05-03 09:56:01--  https://archive.ics.uci.edu/ml/machine-learning-databases/00278/SEMG_DB1.rar
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14440981 (14M) [application/x-httpd-php]
Saving to: ‘SEMG_DB1.rar’

SEMG_DB1.rar        100%[===================>]  13.77M  7.33MB/s    in 1.9s    

2023-05-03 09:56:03 (7.33 MB/s) - ‘SEMG_DB1.rar’ saved [14440981/14440981]



In [3]:
!unrar e  SEMG_DB1.rar


UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from SEMG_DB1.rar

Extracting  10Amar.log                                                     0%  OK 
Extracting  10Apie.log                                                     0%  1%  OK 
Extracting  10Asen.log                                                     1%  OK 
Extracting  11Amar.log                                                     1%  2%  OK 
Extracting  11Apie.log                                                     2%  OK 
Extracting  11Asen.log                                                     3%  OK 
Extracting  1Amar.log                                                      3%  OK 
Extracting  1Apie.log                                                      4%  OK 
Extracting  1Asen.log                                                      4%  OK 
Extracting  2Amar.log                             

In [4]:
!mkdir data

In [5]:
!mkdir data/A
!mkdir data/N

In [6]:
!mv [1-9]A*.txt data/A
!mv [1-9]N*.txt data/N

In [7]:
!mv 1[0-9]A*.txt data/A
!mv 1[0-9]N*.txt data/N

In [8]:
!ls data/A

10Amar.txt  11Asen.txt	2Apie.txt  4Amar.txt  5Asen.txt  7Apie.txt  9Amar.txt
10Apie.txt  1Amar.txt	2Asen.txt  4Apie.txt  6Amar.txt  7Asen.txt  9Apie.txt
10Asen.txt  1Apie.txt	3Amar.txt  4Asen.txt  6Apie.txt  8Amar.txt  9Asen.txt
11Amar.txt  1Asen.txt	3Apie.txt  5Amar.txt  6Asen.txt  8Apie.txt
11Apie.txt  2Amar.txt	3Asen.txt  5Apie.txt  7Amar.txt  8Asen.txt


In [9]:
!ls data/N

10Nmar.txt  11Nsen.txt	2Npie.txt  4Nmar.txt  5Nsen.txt  7Npie.txt  9Nmar.txt
10Npie.txt  1Nmar.txt	2Nsen.txt  4Npie.txt  6Nmar.txt  7Nsen.txt  9Npie.txt
10Nsen.txt  1Npie.txt	3Nmar.txt  4Nsen.txt  6Npie.txt  8Nmar.txt  9Nsen.txt
11Nmar.txt  1Nsen.txt	3Npie.txt  5Nmar.txt  6Nsen.txt  8Npie.txt
11Npie.txt  2Nmar.txt	3Nsen.txt  5Npie.txt  7Nmar.txt  8Nsen.txt


In [11]:
def read_folder(path): 
  folder_con = []
  file_name = []
  n = 1
  for file in os.listdir(path):
      # Check whether file is in text format or not
      if file.endswith(".txt"):
          file_path = f"{path}/{file}"
          folder_con.append(pd.read_csv(file_path, sep='\t', skiprows=list(range(7)), header=None).dropna().drop())
          file_name.append(file_path)
          n = n + 1
          
  return folder_con, file_name

In [13]:
A_folder_con, A_name = read_folder('data/A')
N_folder_con, N_name = read_folder('data/N')


In [15]:
A_folder_con[1]

,0,1,2,3,4
0,-0.0150,-0.0023,0.0045,0.0292,11.4
1,-0.0113,0.0015,0.0067,-0.0346,9.9
2,-0.0165,0.0015,-0.0008,-0.0825,9.8
3,-0.0181,0.0007,-0.0053,-0.1140,9.3
4,-0.0143,-0.0068,-0.0083,-0.1373,8.9
...,...,...,...,...,...
15720,-0.0083,-0.0128,0.1125,0.0255,1.6
15721,-0.0046,-0.0098,0.1155,0.0097,1.5
15722,0.0007,-0.0038,0.1155,0.0097,1.5
15723,0.0007,0.0022,0.1125,0.0277,1.4
